Praktikum 2


Langkah 1: Persiapan Environment


Langkah pertama adalah menyiapkan struktur direktori kerja dan memastikan file model tersedia.

1. Buat folder baru di komputer Anda dengan nama daynight-project.

2. Cari file model (day_night_model.h5) dan scaler (scaler.pkl) hasil training dari praktikum sebelumnya.

3. Pindahkan kedua file tersebut ke dalam folder daynight-project.

4. Buka terminal (Git Bash atau CMD), lalu arahkan direktori ke folder tersebut.

Langkah 2: Konfigurasi Server (app.py)


Diperlukan aplikasi web sederhana menggunakan Flask untuk menangani request gambar dari pengguna.

1. Buat file baru bernama app.py di dalam folder proyek.

2. Salin kode berikut ke dalamnya (kode ini memuat model dan melakukan preprocessing HOG yang sama persis dengan saat training):